In [2]:
from aleph_alpha_client import Client, Prompt, CompletionRequest, CompletionResponse
from scipy import spatial
import numpy as np
import os
from dotenv import load_dotenv

from langchain.llms import AlephAlpha

### Step 0: Define some helper functions

In [34]:
load_dotenv()

# for using the Aleph Alpha API
client = Client(token=os.getenv("AA_TOKEN"))

# for using LangChain
aleph_alpha = AlephAlpha(aleph_alpha_api_key=os.getenv("AA_TOKEN"), model="luminous-extended")

#### Step 1: Use the completion function of the API


In [11]:
# using the full-fledged AA API for a simple completion

# Define the prompt
prompt = Prompt.from_text("An apple a day")

# Create a completion request with parameters
request = CompletionRequest(prompt=prompt, 
                            maximum_tokens=100, 
                            temperature=0.5, 
                            stop_sequences=["\n"])

# Send the request to the API
response = client.complete(request=request, model="luminous-extended")

# Get the completion
completion = response.completions[0].completion
print(completion)

 keeps the doctor away, but if you have a heart condition, it could also save your life.


In [25]:
# using LangChain for a simple completion

# define the prompt
prompt = "An apple a day"

# define the parameters
params = {
    "temperature": 0.5,
    "model": "luminous-extended",
    "maximum_tokens": 100
}

# define the model
aleph_alpha = AlephAlpha(aleph_alpha_api_key=os.getenv("AA_TOKEN"), **params)

# get the completion
response = aleph_alpha(prompt=prompt, stop=["\n"])
print(response)

 keeps the doctor away, but there are other options for those who don’t like apples, or have other dietary restrictions.


#### Step 2: Write a few shot prompt that generates keywords

In [31]:
texts = [
    "Pinocchio was not a boy, but a wooden puppet. He was made by a carpenter named Geppetto. He was a very naughty puppet. He was always getting into trouble. He was always lying.",
    "Most commonly associated with the polar regions, permafrost is soil and rocky material that stays frozen continuously for at least two years. Normally it lies beneath an active layer that melts and freezes depending on the season. Less well known is that permafrost can also be found on steep mountain walls.",
    "Toheroa are a clam that grow as large as a human hand and burrow in intertidal sands on just a handful of epic surf-swept beaches – mainly on the west coast of New Zealand's North Island, but also in isolated colonies at places like Oreti, a beach at the nation's southern tip.",
    "On the neighbourhood's southern edge, cutting through Queens like a backbone, is Roosevelt Avenue. Here, conversations don't stop when the 7 train rattles overhead, they just get louder. Phone repair shops run by Tibetans with makeshift shrines displayed between plastic iPhone covers abut Latin American bakeries churning out pillowy almojábanas (Colombian cheese bread) and crispy empanadas."
]

# TODO write a function that takes a text and returns a list of keywords

def get_keywords(text):
    # TODO Write a prompt that generates keywords for any text
    # Tipp: Use few-shot learning
    
    # 1. define the prompt
    prompt = Prompt.from_text(f"""Task: Extract topic from the text.\nText:{text}\ntopic:""")   
    
    # 2. define the completion request 
    request = CompletionRequest(prompt=prompt, 
                                    maximum_tokens=32, 
                                    temperature=0, 
                                    stop_sequences=["\n"])
    
    # 3. send the request to the API
    response = client.complete(request=request, model="luminous-base")
    
    # 4. get the completion
    completion = response.completions[0].completion
    return completion

def get_keywords_langchain(text):
    # TODO Write a prompt that generates keywords for any text
    # Tipp: Use few-shot learning
    
    # 1. define the prompt
    prompt = f"""Task: Extract topic from the text.\nText:{text}\ntopic:"""
    
    # 2. define the parameters
    params = {
        "temperature": 0.5,
        "model": "luminous-base",
        "maximum_tokens": 100
    }

    # 3. define the model
    aleph_alpha = AlephAlpha(aleph_alpha_api_key=os.getenv("AA_TOKEN"), **params)
    
    # 4. get the completion
    response = aleph_alpha(prompt=prompt, stop=["\n"])
    return response
    

for text in texts:
    keywords = get_keywords(text)
    print(keywords)
    keywords_lang = get_keywords_langchain(text)
    print(keywords_lang)

 Pinocchio
 Pinocchio
permafrost
permafrost
Toheroa
Toheroa
 Roosevelt Avenue
 Roosevelt Avenue


### Let's use Langchain to run a chain

In [33]:
# use a chain from langchain to generate a text

from langchain import PromptTemplate, LLMChain

template = """Q: {question}

A:"""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm = AlephAlpha(model="luminous-extended", maximum_tokens=20, stop_sequences=["Q:"], aleph_alpha_api_key=os.getenv("AA_TOKEN"))

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What is AI?"

llm_chain.run(question)

' Artificial Intelligence (AI) is the simulation of human intelligence processes by machines, especially computer systems.\n'